In [ ]:
!pip install tensorflow opencv-python matplotlib numpy gdown

import os
import requests

# Create dataset directories
categories = ["cat", "dog"]
os.makedirs("dataset/cat", exist_ok=True)
os.makedirs("dataset/dog", exist_ok=True)

# Image URLs (from Unsplash)
image_urls = {
    "cat": [
        "https://images.unsplash.com/photo-1518791841217-8f162f1e1131",
        "https://images.unsplash.com/photo-1574158622682-e40e69881006"
    ],
    "dog": [
        "https://images.unsplash.com/photo-1560807707-8cc77767d783",
        "https://images.unsplash.com/photo-1560807707-8cc77767d783"
    ]
}

# Download images
for category, urls in image_urls.items():
    for i, url in enumerate(urls):
        img_data = requests.get(url).content
        with open(f"dataset/{category}/{category}{i+1}.jpg", "wb") as handler:
            handler.write(img_data)

print("✅ Sample dataset downloaded and saved!")

In [ ]:
# Alternative: Download dataset from Google Drive if available
!gdown --folder --id 1B19RWODijlUlXEdvLKxjcmlg-BiJcbWE -O /content/shared_folder
print("✅ Dataset downloaded from Google Drive!")

In [ ]:
import tensorflow as tf
import cv2
import numpy as np
import matplotlib.pyplot as plt

# Load a pre-trained model (MobileNetV2 for image classification)
model = tf.keras.applications.MobileNetV2(weights="imagenet")

# Function to preprocess image
def preprocess_image(image_path):
    img = cv2.imread(image_path)
    img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
    img = cv2.resize(img, (224, 224))
    img = np.expand_dims(img, axis=0)
    img = tf.keras.applications.mobilenet_v2.preprocess_input(img)
    return img

# Function to predict object in image
def predict_object(image_path):
    img = preprocess_image(image_path)
    predictions = model.predict(img)
    decoded_predictions = tf.keras.applications.mobilenet_v2.decode_predictions(predictions, top=3)[0]

    for i, (imagenet_id, label, score) in enumerate(decoded_predictions):
        print(f"{i+1}: {label} ({score:.2f})")

    # Display image
    img_display = cv2.imread(image_path)
    img_display = cv2.cvtColor(img_display, cv2.COLOR_BGR2RGB)
    plt.imshow(img_display)
    plt.axis("off")
    plt.show()

# Example usage
sample_image = "dataset/cat/cat1.jpg"  # Change this based on dataset availability
predict_object(sample_image)